In [4]:
import os
import win32com.client as win32
import re
import pandas as pd
import PyPDF2

In [5]:
#Directorio de trabajo
os.chdir("C:/Users/claudio.pacheco/Desktop/ejemplos2")

In [6]:
#Identificar todos los archivos pdf del directorio
archivos = os.listdir()
pdf_files = [f for f in archivos if f.endswith('.pdf')]
pdf_files

['01 Ene.pdf', '12 Dic.pdf']

In [7]:
#Crear objeto de word
word = win32.gencache.EnsureDispatch('Word.Application')
word.Visible = False

In [8]:
for pdf_file in pdf_files:
    doc = word.Documents.Open(os.path.join(os.getcwd(), pdf_file))
    doc.SaveAs(os.path.join(os.getcwd(), pdf_file[:-4] + '_edit.pdf'), FileFormat=17)
    doc.Close()

# Quit Word to free up resources
word.Quit()


In [9]:
editables= [f for f in os.listdir() if f.endswith('_edit.pdf')]
editables

['01 Ene_edit.pdf', '12 Dic_edit.pdf']

In [10]:
dfs = []  # Lista para guardar los dataframes

for editable in editables:
    pdfFileObj = open(os.path.join(os.getcwd(), editable), 'rb')
    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    texto = pdfReader.pages[0].extract_text()
    
    # Extraer número de oficio
    oficio_start_index = texto.find("No. ")
    oficio_end_index = texto.find("Ciudad de México")
    if oficio_start_index != -1 and oficio_end_index != -1:
        oficio_text = texto[oficio_start_index:oficio_end_index + len("Ciudad de México")]
        oficio_text = re.sub(' +', ' ', oficio_text).replace('\n', ' ').replace('Ciudad de México', '').replace('No.', '').strip()
    
    # Extraer fecha
    pattern = r'\b(\d{1,2}) de ([a-zA-Z]+) de (\d{4})\b'
    match = re.search(pattern, texto)
    if match:
        day, month, year = match.groups()
        date = f"{day} {month} {year}"

    # Extraer concepto and monto
    start_index = texto.find("Concepto")
    end_index = texto.find("Total")
    if start_index != -1 and end_index != -1:
        extracted_text = texto[start_index:end_index + len("Total")]
        clean_text = re.sub(r'\'|,', '', extracted_text)
        numbers = re.findall(r'\d+', clean_text)
        df = pd.DataFrame(numbers, columns=['monto'])
        conceptos = ["FGP", "FFM", "IEPS"]
        df['concepto'] = conceptos
        df['oficio'] = oficio_text
        df['fecha'] = date
        df['monto'] = df['monto'].astype(int)
        dfs.append(df)

# Pegar dfs
final_df = pd.concat(dfs, ignore_index=True)

In [11]:
final_df

,monto,concepto,oficio,fecha
0,667493503,FGP,351 -A-DGPA -C-267,19 enero 2018
1,43139478,FFM,351 -A-DGPA -C-267,19 enero 2018
2,9007129,IEPS,351 -A-DGPA -C-267,19 enero 2018
3,712355256,FGP,351 -A-DCPA -C-5551,19 diciembre 2018
4,47525248,FFM,351 -A-DCPA -C-5551,19 diciembre 2018
5,12951366,IEPS,351 -A-DCPA -C-5551,19 diciembre 2018


In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   monto     6 non-null      int32 
 1   concepto  6 non-null      object
 2   oficio    6 non-null      object
 3   fecha     6 non-null      object
dtypes: int32(1), object(3)
memory usage: 296.0+ bytes
